In [21]:
#########################################################################
# Author: Dan Betea                                                     #
# (C) December 2022                                                     #
# License: CC BY-SA 4.0                                                 #
# License description: https://creativecommons.org/licenses/by-sa/4.0/  #
#########################################################################

# LUE patterns 2: linear regression with naive train/test split

Consider the Laguerre-$\alpha$ Unitary Ensemble (LUE-$\alpha$) distribution on ordered tuples of $N$ positive real numbers $(\lambda_1 < \dots < \lambda_N).$ That is, consider the probability measure

$$P(\lambda_1, \dots, \lambda_N)d \lambda_1 \dots d \lambda_N \propto \prod_{1 \leq i < j \leq N} (\lambda_i - \lambda_j)^2 \prod_{1 \leq i \leq N} \lambda_i^{\alpha-1} e^{-\lambda_i} d \lambda_i$$

where $\alpha > 0$ (notice the somewhat less standard $\alpha$ convention we use for LUE). See [this Wikipedia article](https://en.wikipedia.org/wiki/Complex_Wishart_distribution) for the motivation behind this distribution and how it comes about when studying covariance matrices (see in particular the Eigenvalues section).

We do linear regression (in Python, using Scikit-learn) on 

$$(\log i, \log E \lambda_i^s)$$

for $i$ in a certain interval like $1, \dots, 10; 1, \dots, \log N$ or more generally $m_0, \dots, m_0 + M - 1$. Here $\lambda_i$ the $i$-th lowest eigenvalue of the LUE-$\alpha$ ensemble and $s$ is a real number, taken negative for convergence. 

We consider several cases:

- $m_0 = 1, M = 15$, $\alpha = 4, s = -2$, $N \in \{1000, 5000, 10000\}$, and 1000 total samples

and we additionally consider two cases for splitting the data into training and testing: 

- first we try 60%-40%, and
- then we try 80%-20%.

One we have variables $y_i^{train}$ and $y_i^{test}$, computing the $R^2$ coefficient (metric) becomes a matter of choice. We compute three such coefficients: on the training set only, on the test set only, and out-of-sample on the train and test set as follows:

$$R^2_{train} = 1 - \frac{\sum_i (y_i^{train} - \hat{y}_i^{train})^2}{\sum_i (y_i^{train} - \bar{y}^{train})^2}, \qquad
R^2_{test} = 1 - \frac{\sum_i (y_i^{test} - \hat{y}_i^{test})^2}{\sum_i (y_i^{test} - \bar{y}^{test})^2}, \qquad
R^2_{oos} = 1 - \frac{\sum_i (y_i^{test} - \hat{y}_i^{test})^2}{\sum_i (y_i^{test} - \bar{y}^{train})^2}$$

and the reason for $R^2_{oos}$ is the following: we check the test set performance against the benchmark *non-prediction model* we see on the training/fitted set, which is the model which outputs the mean of the training set $\bar{y}^{train}$ no matter the data.

**Important remark:** The dependent variable $x = (\log i)_{m_0 \leq i \leq m_0+M}$ is deterministic, so $\hat{y}^{train} = \hat{y}^{test}$ using our setup.

**Some other remarks:**

- array indexing starts at 0 by default in Python
- the data files, containing iid samples $(\lambda_1 < \dots < \lambda_N)$, are assumed to be in the same directory as the notebook, and be in the correct format
- steps below can be automated; for this exploratory notebook they are not

In [22]:
# importing the necessary packages
import numpy as np                 # for linear algebra and loading from files
from sklearn import linear_model   # for linear regression

First we set up the parameters independent of the data split: the values of $N$, $\alpha$, $s$, $m_0, M$, and the $x$ vector.

In [23]:
Ns = [1000, 5000, 10000]   # matrix sizes to be considered
s = -2.0                   # point where to compute E \lambda_i^s
alpha_str = "4.00"
alpha = float(alpha_str)   # = 4.0, Laguerre parameter alpha-1 
m0 = 0                     # regression starts at \lambda_{m_0+1}
M = 15                     # regression ends with \lambda_{m_0+M} 
range_ms = range(m0, m0+M) # index range for regression

# building up dictionary of filenames (data is read from these files)
filename_dict = {}
for N in Ns:
    filename = f"LUE_N_{N}_alpha_{alpha_str}.txt"
    filename_dict[str(N)] = filename
    
# build the x variables for regression
# x = log i for m0+1 <= i <= m0+M (+1 because of Python 0-indexed arrays) 
# reshape into column vectors needed
log_ms = np.log(np.arange(m0+1, m0+M+1)).reshape(-1, 1)
print("x = log i = ")
print(log_ms.flatten())
# print("\n"+73*"-")

x = log i = 
[0.         0.69314718 1.09861229 1.38629436 1.60943791 1.79175947
 1.94591015 2.07944154 2.19722458 2.30258509 2.39789527 2.48490665
 2.56494936 2.63905733 2.7080502 ]


### First case: 60-40 split

Next we set up the train and test data, namely the $y$ vectors corresponding to train and test sets.

In [4]:
# build up a dictionary of the data for each N and
# load up the corresponding matrix A, only M columns: m0, ..., m0+M-1 and
# build the y variables for regression into a dictionary, one vector per N
# (same reshape needed as above)
log_expectations_train_dict = {}
log_expectations_test_dict = {}
for N in Ns:
    A = np.loadtxt(filename_dict[str(N)], usecols=range_ms)
    A = A[:1000, :]              # take only 1000 samples, for consistency
    A_train = A[:600, :]         # 80 % for training
    A_test = A[600:, :]          # 20 % test
    print()
    print(f"N = {N}\n")
    print("A_train = eigenvalue sample matrix (iid / line) = \n")
    print(A_train)
    print()
    print(f"shape of A_train is: {A.shape}\n")
    # below: axis = 0 means we sum over rows, i.e. take the average of each column (iid eigenval sample)
    # build the y variable, for training
    log_expectations_train = np.log(np.mean(A_train**s, axis=0)).reshape(-1, 1) 
    log_expectations_train_dict[str(N)] = log_expectations_train
    # build the y variable, for testing
    log_expectations_test = np.log(np.mean(A_test**s, axis=0)).reshape(-1, 1) 
    log_expectations_test_dict[str(N)] = log_expectations_test
    print("log E \lambda_i^s (train) = \n")
    print(log_expectations_train.flatten())
    print("\n"+73*"-")


N = 1000

A_train = eigenvalue sample matrix (iid / line) = 

[[0.00914148 0.01421346 0.02985021 ... 0.42833932 0.45033342 0.52411581]
 [0.01065387 0.016576   0.02827056 ... 0.42273216 0.51435102 0.55804161]
 [0.00897166 0.01592701 0.0399726  ... 0.42665176 0.51484552 0.53870744]
 ...
 [0.0043258  0.01206655 0.03115058 ... 0.45655002 0.57092822 0.67062551]
 [0.00438205 0.03887275 0.05732078 ... 0.50094824 0.55490721 0.66749578]
 [0.0058209  0.01021484 0.035881   ... 0.46072031 0.55426236 0.60527526]]

shape of A_train is: (1000, 15)

log E \lambda_i^s (train) = 

[10.60487598  8.10004521  6.73538035  5.75510236  4.9694163   4.32724725
  3.77248462  3.28860453  2.84765712  2.45641598  2.11183603  1.7882116
  1.47478295  1.201755    0.94275976]

-------------------------------------------------------------------------

N = 5000

A_train = eigenvalue sample matrix (iid / line) = 

[[0.00114    0.00280814 0.00431431 ... 0.0926519  0.11608572 0.11824966]
 [0.00111132 0.00349919 0.0055273  

#### Training (followed by testing)

The actual regression goes down below. First the training phase to get the regression line.

**Note:** The actual $R^2$ coefficient displayed below is obviously computed on the training set, and expected to be big. It is still nevertheless *quite high*.

In [27]:
# doing the actual regression
reg_dict = {} # dictionary to hold the regressors
# below: dictionary to hold the coefficients of the regression in the form
# str(N): {"b0": ..., "b1": ..., "r2": ...}
# with b0 = intercept, b1 = slope, r2 = R squared
coeff_dict = {str(N) : {} for N in Ns} 
for N in Ns:
    print(f"\n N = {N}\n")
    log_expectations_train = log_expectations_train_dict[str(N)]
    log_expectations_test = log_expectations_test_dict[str(N)]
    reg_dict[str(N)] = linear_model.LinearRegression()
    reg_dict[str(N)].fit(log_ms, log_expectations_train) # linear fit
    reg = reg_dict[str(N)]
    
    # computing R^2 for train and test (naive)
    r2_train = reg.score(log_ms, log_expectations_train)
    r2_test = reg.score(log_ms, log_expectations_test)
    # computing the R^2 out-of-sample coefficient
    SS_tot_train_test = np.sum((log_expectations_test - np.mean(log_expectations_train))**2)
    SS_res_test = np.sum((log_expectations_test - (reg.intercept_.item() + reg.coef_.item() * log_ms))**2)
    r2_oos = 1 - SS_res_test/SS_tot_train_test
     
    print(f" slope:             {reg.coef_.item()}")
    print(f" intercept:         {reg.intercept_.item()}")
    print(f" R squared (train):  {r2_train}")
    print(f" R squared (test):   {r2_test}")
    print(f" R squared (oos):    {r2_oos}")
    coeff_dict[str(N)]["b0"] = reg.coef_.item()
    coeff_dict[str(N)]["b1"] = reg.intercept_.item()
    coeff_dict[str(N)]["r2_train"] = r2_train
    coeff_dict[str(N)]["r2_test"] = r2_test
    coeff_dict[str(N)]["r2_oos"] = r2_oos
    print("\n"+73*"-")


 N = 1000

 slope:             -3.5565342748568156
 intercept:         10.640085997010413
 R squared (train):  0.9997252840928547
 R squared (test):   0.9996010133806793
 R squared (oos):    0.9996010244247339

-------------------------------------------------------------------------

 N = 5000

 slope:             -3.524984873534735
 intercept:         13.780306377117752
 R squared (train):  0.9996525781321135
 R squared (test):   0.9995437008969466
 R squared (oos):    0.9995437012847317

-------------------------------------------------------------------------

 N = 10000

 slope:             -3.560837784875215
 intercept:         15.241066745513617
 R squared (train):  0.9997641013695571
 R squared (test):   0.9986867445554635
 R squared (oos):    0.9986867990895595

-------------------------------------------------------------------------


### Second case: 80-20 split

In [25]:
# build up a dictionary of the data for each N and
# load up the corresponding matrix A, only M columns: m0, ..., m0+M-1 and
# build the y variables for regression into a dictionary, one vector per N
# (same reshape needed as above)
log_expectations_train_dict_2 = {}
log_expectations_test_dict_2 = {}
for N in Ns:
    A = np.loadtxt(filename_dict[str(N)], usecols=range_ms)
    A = A[:1000, :]              # take only 1000 samples, for consistency
    A_train = A[200:, :]         # 80 % for training
    A_test = A[:200, :]          # 20 % test
    print()
    print(f"N = {N}\n")
    print("A_train = eigenvalue sample matrix (iid / line) = \n")
    print(A_train)
    print()
    print(f"shape of A_train is: {A.shape}\n")
    # below: axis = 0 means we sum over rows, i.e. take the average of each column (iid eigenval sample)
    # build the y variable, for training
    log_expectations_train = np.log(np.mean(A_train**s, axis=0)).reshape(-1, 1) 
    log_expectations_train_dict_2[str(N)] = log_expectations_train
    # build the y variable, for testing
    log_expectations_test = np.log(np.mean(A_test**s, axis=0)).reshape(-1, 1) 
    log_expectations_test_dict_2[str(N)] = log_expectations_test
    print("log E \lambda_i^s (train) = \n")
    print(log_expectations_train.flatten())
    print("\n"+73*"-")


N = 1000

A_train = eigenvalue sample matrix (iid / line) = 

[[0.00405837 0.01576241 0.03433577 ... 0.51706077 0.57475493 0.6476627 ]
 [0.00648518 0.01320577 0.03985386 ... 0.49188188 0.5384115  0.63590799]
 [0.0109845  0.03155448 0.04758467 ... 0.47631312 0.51299513 0.60330846]
 ...
 [0.0104201  0.01963166 0.03846435 ... 0.43218855 0.51115892 0.55671524]
 [0.00582622 0.00984355 0.03220857 ... 0.43013025 0.51010797 0.61515227]
 [0.01404306 0.02702059 0.04454848 ... 0.43967195 0.58155765 0.72504212]]

shape of A_train is: (1000, 15)

log E \lambda_i^s (train) = 

[10.6138792   8.09176491  6.72208319  5.73931204  4.96804947  4.33075151
  3.76904102  3.28213565  2.84679669  2.45226633  2.10730741  1.78289361
  1.47532656  1.19744723  0.94050505]

-------------------------------------------------------------------------

N = 5000

A_train = eigenvalue sample matrix (iid / line) = 

[[0.00250476 0.00369888 0.00814249 ... 0.09167625 0.11054006 0.12709592]
 [0.00116709 0.00259737 0.00807019

#### Training (followed by testing)

In [28]:
# doing the actual regression
reg_dict_2 = {} # dictionary to hold the regressors
# below: dictionary to hold the coefficients of the regression in the form
# str(N): {"b0": ..., "b1": ..., "r2": ...}
# with b0 = intercept, b1 = slope, r2 = R squared
coeff_dict_2 = {str(N) : {} for N in Ns}
for N in Ns:
    print(f"\n N = {N}\n")
    log_expectations_train = log_expectations_train_dict_2[str(N)]
    log_expectations_test = log_expectations_test_dict_2[str(N)]
    reg_dict_2[str(N)] = linear_model.LinearRegression()
    reg_dict_2[str(N)].fit(log_ms, log_expectations_train) # linear fit
    reg = reg_dict_2[str(N)]
    
    # computing R^2 for train and test (naive)
    r2_train = reg.score(log_ms, log_expectations_train)
    r2_test = reg.score(log_ms, log_expectations_test)
    # computing the R^2 out-of-sample coefficient
    SS_tot_train_test = np.sum((log_expectations_test - np.mean(log_expectations_train))**2)
    SS_res_test = np.sum((log_expectations_test - (reg.intercept_.item() + reg.coef_.item() * log_ms))**2)
    r2_oos = 1 - SS_res_test/SS_tot_train_test
     
    print(f" slope:             {reg.coef_.item()}")
    print(f" intercept:         {reg.intercept_.item()}")
    print(f" R squared (train):  {r2_train}")
    print(f" R squared (test):   {r2_test}")
    print(f" R squared (oos):    {r2_oos}")
    coeff_dict[str(N)]["b0"] = reg.coef_.item()
    coeff_dict[str(N)]["b1"] = reg.intercept_.item()
    coeff_dict[str(N)]["r2_train"] = r2_train
    coeff_dict[str(N)]["r2_test"] = r2_test
    coeff_dict[str(N)]["r2_oos"] = r2_oos
    print("\n"+73*"-")


 N = 1000

 slope:             -3.556985804860991
 intercept:         10.637124810312272
 R squared (train):  0.9997278908057867
 R squared (test):   0.9993046075393165
 R squared (oos):    0.9993046167559675

-------------------------------------------------------------------------

 N = 5000

 slope:             -3.5184585909075796
 intercept:         13.765136769385732
 R squared (train):  0.9996063762935679
 R squared (test):   0.9996495122867071
 R squared (oos):    0.9996495175602309

-------------------------------------------------------------------------

 N = 10000

 slope:             -3.525326475682311
 intercept:         15.168079151939518
 R squared (train):  0.9996381045595585
 R squared (test):   0.9996484001983278
 R squared (oos):    0.9996484010579837

-------------------------------------------------------------------------


## Miniconclusion

Even with a train-test split of 60-40 or 80-20, the relation looks solidly linear. Note one thing though: our variable $x$ vector never changes, it remains constant.